## Initial configuration

In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import cv2
import gc
from tqdm import tqdm,trange

import torch
from torchsummary import summary
from PIL import Image
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.optim as optim
import matplotlib.pyplot as plt

import urllib.request
import zipfile

In [2]:
#Usefull for better outputs visualization
PURPLE = "\033[95m"
CYAN = "\033[96m"
DARKCYAN = "\033[36m"
BLUE = "\033[94m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
END = "\033[0m"

### Project settings

In [3]:
data_dir_unsplitted = "dataset/unsplitted"
save_dir_splitted = "dataset/splitted"
data_dir = "dataset/splitted"
models_dir ="model/"

learning_rate_e = 0.0001
learning_rate_d = 0.0001
learning_rate_c = 0.0001
num_epochs = 50

ALREADY_SPLITTED = True

#to be implemented:

LOAD_DATASET = True #if true "ALREADY_SPLITTED" is ignored
DATASET_ALREADY_DOWNLOADED = True 

LOAD_MODEL = False 
MODEL_ALREADY_DOWNLOADED = True 

## Dataset preparation

### Dataset splitting and data augmentation

In [4]:
classes = ["CLL", "FL", "MCL"]
id_to_name = {0:"CLL",1:"FL",2:"MCL"}

if(not ALREADY_SPLITTED or not LOAD_DATASET):
    data = []
    labels = []
    for i, cl in enumerate(classes):
        path = os.path.join(data_dir_unsplitted, cl)
        images = os.listdir(path)
        for img in tqdm(images,desc=cl):
            img_path = os.path.join(path, img)
            img = cv2.imread(img_path).astype(np.uint8)

            #DATA AUGMENTATION

            #flipping the image
            flipped_img = np.fliplr(img)
            #gaussian noise
            noise = np.random.normal(0, 5, img.shape).astype(np.uint8)
            noisy_img = img + noise

            #adding the data and the labels to the lists
            data.append(flipped_img)
            labels.append(i)
            data.append(noisy_img)
            labels.append(i)
            data.append(img)
            labels.append(i) 


    data = np.array(data)
    labels = np.array(labels)


    train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)


    train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.125, random_state=42)

    print("Train samples:", len(train_data))
    print("Test samples:", len(test_data))
    print("Validation samples:", len(val_data))

### Saving the splitted dataset

In [5]:
#SAVING THE DATASET
if(not ALREADY_SPLITTED or not LOAD_DATASET):
    #TRAIN
    os.makedirs(os.path.join(save_dir_splitted,"train","images"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"train","labels"),exist_ok=True)

    np.save(os.path.join(save_dir_splitted,"train","labels","train.npy"),train_labels)#labels

    for i, image in enumerate(tqdm(train_data,desc = "Train")):                       #images
        file_name = str(i)+"_train"+".png"
        path = os.path.join(save_dir_splitted,"train","images",file_name)
        cv2.imwrite(path,image.astype(np.float64))

    #TEST
    os.makedirs(os.path.join(save_dir_splitted,"test","images"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"test","labels"),exist_ok=True)

    np.save(os.path.join(save_dir_splitted,"test","labels","test.npy"),test_labels)#labels

    for i, image in enumerate(tqdm(test_data,desc = "Test")):                      #images
        file_name = str(i)+"_test"+".png"
        path = os.path.join(save_dir_splitted,"test","images",file_name)
        cv2.imwrite(path,image.astype(np.float64))

    #VALIDATION
    os.makedirs(os.path.join(save_dir_splitted,"val","images"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"val","labels"),exist_ok=True)

    np.save(os.path.join(save_dir_splitted,"val","labels","val.npy"),val_labels) #labels

    for i, image in enumerate(tqdm(val_data,desc = "Validation")):               #images
        file_name = str(i)+"_val"+".png"
        path = os.path.join(save_dir_splitted,"val","images",file_name)
        cv2.imwrite(path,image.astype(np.float64))

### Dataloaders definition

Notice that the labels are saved into a numpy array, while the images are read directly from the folder

In [6]:
class HDAdataset(Dataset):
    def __init__(self, root_dir, labels, train = False, transform=None):
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(os.listdir(self.root_dir))

    def __getitem__(self, idx):
        img_name = os.listdir(self.root_dir)[idx]
        img_path = os.path.join(self.root_dir, img_name)

        image = Image.open(img_path)
        image = np.asarray(image).astype(np.uint8)
        
        if self.train:
            image = image + np.random.normal(0, 2, image.shape).astype(np.uint8)
            
        label = np.zeros(3)
        label[self.labels[idx]] = 1
        if self.transform:
            image = self.transform((image/255.).astype(np.float32))

        return image, label

In [7]:
if(LOAD_DATASET and not DATASET_ALREADY_DOWNLOADED):
    print("download the dataset")
    urllib.request.urlretrieve("", "dataset.zip")
    with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
        zip_ref.extractall("./dataset/")


#convert to tensor each image
transform = transforms.Compose([transforms.ToTensor()])    

#TRAIN DATASET AND DATALOADER
train_dir = os.path.join(data_dir,"train","images")#images

train_labels = np.load(os.path.join(data_dir,"train","labels","train.npy"))#labels

train_dataset = HDAdataset(train_dir,train_labels,train = True, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=3, shuffle=True,drop_last=True)

#TEST DATASET AND DATALOADER
test_dir = os.path.join(data_dir,"test","images")#images

test_labels = np.load(os.path.join(data_dir,"test","labels","test.npy"))#labels

test_dataset = HDAdataset(test_dir,test_labels,train = False,transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=3, shuffle=False)

#TVALIDATION DATASET AND DATALOADER
val_dir = os.path.join(data_dir,"val","images")#images

val_labels = np.load(os.path.join(data_dir,"val","labels","val.npy"))#labels

val_dataset = HDAdataset(val_dir,val_labels,train = False,transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=3, shuffle=False)

## Neural network

In [8]:
torch.cuda.empty_cache()
gc.collect()

33

### Models definition

In [16]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(3,3), stride=(3,3), padding=(1,1)),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Conv2d(in_channels=8, out_channels=1, kernel_size=(3,3), stride=(3,3), padding=(1,1)),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.encoder(x)
        return x
    
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(3,3), padding=(1,1)),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            nn.ConvTranspose2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(3,3), padding=(0,1)),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            nn.ConvTranspose2d(in_channels=16, out_channels=3, kernel_size=(3,3), stride=(1,1), padding=(1,2),dilation=2,output_padding=(0,1)),
            nn.Sigmoid()
        )


    def forward(self, x):
        x = self.decoder(x)
        return x

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(BOLD+"You are working on: "+END+str(device))

You are working on: cuda


### Loading of the models

In [18]:
if(LOAD_MODEL and not MODEL_ALREADY_DOWNLOADED):
    print("download the model")

In [19]:
if(not LOAD_MODEL):
    encoder = Encoder().to(device)
    decoder = Decoder().to(device)
    classifier = Classifier().to(device)
    
else:
    print("load the models from the folder")

### Training 

In [20]:
def test(dataloader,name):
    
    losses_ae = []
    losses_c = []
    count_correct = 0
    count_input = 0

    encoder.eval()
    decoder.eval()
    classifier.eval()
    for data in tqdm(dataloader,desc = name +" set"):

        img, labels = data
        img = img.to(device)
        labels = labels.to(device)


        compressed_img = encoder(img)
        rec_img = decoder(compressed_img)
        pred_labels = classifier(compressed_img)

        classifier_loss = criterion_classifier(pred_labels, labels)
        ae_loss= criterion_ae(img,rec_img)


        losses_ae.append(ae_loss.detach().cpu().numpy())
        losses_c.append(classifier_loss.detach().cpu().numpy())

        count_input+=pred_labels.detach().cpu().numpy().shape[0]
        count_correct+=np.sum((labels.to("cpu").detach().numpy().argmax(axis=1)==pred_labels.to("cpu").detach().numpy().argmax(axis=1)))
        del img
        del labels
        torch.cuda.empty_cache()
        gc.collect();

    # Print loss at end of each epoch
    accuracy= (count_correct/count_input)*100.
    print(name,' Autoencoder Loss: %.4f, Classifier Loss: %.4f, Accuracy: %.4f in %d input samples\n'
      % (np.mean(losses_ae), np.mean(losses_c),accuracy,count_input))
    
    return np.mean(losses_ae),np.mean(losses_c),accuracy

In [24]:
criterion_classifier = nn.CrossEntropyLoss()
criterion_ae = nn.MSELoss()

def train(train_dataloader,val_dataloader):
    optimizer_e = optim.Adam(encoder.parameters(), lr=learning_rate_e,weight_decay=1e-2)
    optimizer_d = optim.Adam(decoder.parameters(), lr=learning_rate_d)
    optimizer_c = optim.Adam(classifier.parameters(), lr=learning_rate_c,weight_decay=1e-2)
    
    ###to be inplemented#####
    best_epoch = 0 #(based on the validation accuracy)
    best_accuracy = -np.inf
    
    #for plotting
    train_loss_ae_trend = []
    train_loss_c_trend = []
    train_accuracy_trend =[]
    val_loss_ae_trend = []
    val_loss_c_trend = []
    val_accuracy_trend = []
    ############################################
    
    for epoch in range(1):

        ######training#####
        losses_ae = []
        losses_c = []
        count_correct = 0
        count_input = 0

        encoder.train()
        decoder.train()
        classifier.train()
        for data in tqdm(train_dataloader,desc = "Train Epoch [%d/%d]"%(epoch+1,num_epochs)):



            img, labels = data
            img = img.to(device)
            labels = labels.to(device)


            optimizer_e.zero_grad()
            optimizer_d.zero_grad()
            optimizer_c.zero_grad()


            compressed_img = encoder(img)
            rec_img = decoder(compressed_img)
            pred_labels = classifier(compressed_img)

            classifier_loss = criterion_classifier(pred_labels, labels)
            ae_loss = criterion_ae(img,rec_img)
            total_loss = 10*classifier_loss+ae_loss

#             classifier_loss.backward(retain_graph=True)
#             ae_loss.backward(retain_graph=True)

            total_loss.backward()
            optimizer_e.step()
            optimizer_d.step()
            optimizer_c.step()


            losses_ae.append(ae_loss.detach().cpu().numpy())
            losses_c.append(classifier_loss.detach().cpu().numpy())

            count_input+=pred_labels.detach().cpu().numpy().shape[0]
            count_correct+=np.sum((labels.to("cpu").detach().numpy().argmax(axis=1)==pred_labels.to("cpu").detach().numpy().argmax(axis=1)))
            del img
            del labels
            torch.cuda.empty_cache()
            gc.collect();
            
        accuracy = (count_correct/count_input)*100.
        
        print('TRAIN: Epoch [%d/%d], Autoencoder Loss: %.4f, Classifier Loss: %.4f, Accuracy: %.4f in %d input samples \n'
              % (epoch+1, num_epochs, np.mean(losses_ae), np.mean(losses_c),accuracy,count_input))
        
        train_loss_ae_trend.append(np.mean(losses_ae))
        train_loss_c_trend.append(np.mean(losses_c))
        train_accuracy_trend.append(accuracy)
        
        ######validation######
        ae,c,acc = test(val_dataloader,"VALIDATION")
        
        val_loss_ae_trend.append(ae)
        val_loss_c_trend.append(c)
        val_accuracy_trend.append(acc)
        

        return np.array([train_loss_ae_trend,train_loss_c_trend,train_accuracy_trend],
                [val_loss_ae_trend,val_loss_c_trend,val_accuracy_trend],dtype=np.object_)

In [ ]:
if(not LOAD_MODEL):
    results = train(train_dataloader,val_dataloader)   

Train Epoch [1/50]:  27%|████████████████▌                                            | 71/261 [00:50<02:13,  1.42it/s]

### Test

Notice that the value of the losses and the accuracy for the train set are computed during the training, so they are not very accurate.

In [ ]:
train_dataset_no_noise = HDAdataset(train_dir,train_labels,train = False, transform=transform)
train_dataloader_no_noise = DataLoader(train_dataset_no_noise, batch_size=3, shuffle=False,drop_last=False)

In [ ]:
test(train_dataloader_no_noise,"TRAIN")
test(val_dataloader,"VALIDATION")
test(test_dataloader,"TEST")